In [122]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import torch
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.metrics import log_loss
from category_encoders.ordinal import OrdinalEncoder

In [150]:
from os.path import join
from os import remove as remove_file
import glob
import pickle
import multiprocessing

n_cpus = multiprocessing.cpu_count()

import nni
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PowerTransformer
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
from category_encoders.ordinal import OrdinalEncoder

In [151]:
BASE_DIR = '/Users/HwaLang/Desktop/python/T academy/Kaggle_camp/'

scaler_dict = {
    "minmax": MinMaxScaler,
    "standard": StandardScaler   
}

def select_scaler(name):
    print(f"Select {name} Scaler")
    
    return scaler_dict[name]()

def preprocess(x_train, x_valid, x_test, target):
    # global num_columns, cat_columns

    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()    
    tmp_x_test  = x_test.copy()
# ---------------------------
    tmp_x_train.drop(columns = ['FLAG_MOBIL'], inplace = True)
    tmp_x_valid.drop(columns = ['FLAG_MOBIL'], inplace = True)
    tmp_x_test.drop(columns = ['FLAG_MOBIL'], inplace = True)

    tmp_x_train['DAYS_EMPLOYED'] = tmp_x_train['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)
    tmp_x_valid['DAYS_EMPLOYED'] = tmp_x_valid['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)
    tmp_x_test['DAYS_EMPLOYED'] = tmp_x_test['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)

    feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
    for feat in feats:
        tmp_x_train[feat]=np.abs(tmp_x_train[feat])
        tmp_x_valid[feat]=np.abs(tmp_x_valid[feat])
        tmp_x_test[feat]=np.abs(tmp_x_test[feat])
    
    for df in [tmp_x_train, tmp_x_valid, tmp_x_test]:
        # before_EMPLOYED: 고용되기 전까지의 일수
        df['before_EMPLOYED'] = df['DAYS_BIRTH'] - df['DAYS_EMPLOYED']
        df['income_total_befofeEMP_ratio'] = df['income_total'] / df['before_EMPLOYED']
        df['before_EMPLOYED_m'] = np.floor(df['before_EMPLOYED'] / 30) - ((np.floor(df['before_EMPLOYED'] / 30) / 12).astype(int) * 12)
        df['before_EMPLOYED_w'] = np.floor(df['before_EMPLOYED'] / 7) - ((np.floor(df['before_EMPLOYED'] / 7) / 4).astype(int) * 4)
        
        #DAYS_BIRTH 파생변수- Age(나이), 태어난 월, 태어난 주(출생연도의 n주차)
        df['Age'] = df['DAYS_BIRTH'] // 365
        df['DAYS_BIRTH_m'] = np.floor(df['DAYS_BIRTH'] / 30) - ((np.floor(df['DAYS_BIRTH'] / 30) / 12).astype(int) * 12)
        df['DAYS_BIRTH_w'] = np.floor(df['DAYS_BIRTH'] / 7) - ((np.floor(df['DAYS_BIRTH'] / 7) / 4).astype(int) * 4)

        
        #DAYS_EMPLOYED_m 파생변수- EMPLOYED(근속연수), DAYS_EMPLOYED_m(고용된 달) ,DAYS_EMPLOYED_w(고용된 주(고용연도의 n주차))  
        df['EMPLOYED'] = df['DAYS_EMPLOYED'] // 365
        df['DAYS_EMPLOYED_m'] = np.floor(df['DAYS_EMPLOYED'] / 30) - ((np.floor(df['DAYS_EMPLOYED'] / 30) / 12).astype(int) * 12)
        df['DAYS_EMPLOYED_w'] = np.floor(df['DAYS_EMPLOYED'] / 7) - ((np.floor(df['DAYS_EMPLOYED'] / 7) / 4).astype(int) * 4)

        #ability: 소득/(살아온 일수+ 근무일수)
        df['ability'] = df['income_total'] / (df['DAYS_BIRTH'] + df['DAYS_EMPLOYED'])
        
        #income_mean: 소득/ 가족 수
        df['income_mean'] = df['income_total'] / df['family_size']
        
        #ID 생성: 각 컬럼의 값들을 더해서 고유한 사람을 파악(*한 사람이 여러 개 카드를 만들 가능성을 고려해 begin_month는 제외함)
        df['ID'] = \
        df['child_num'].astype(str) + '_' + df['income_total'].astype(str) + '_' +\
        df['DAYS_BIRTH'].astype(str) + '_' + df['DAYS_EMPLOYED'].astype(str) + '_' +\
        df['work_phone'].astype(str) + '_' + df['phone'].astype(str) + '_' +\
        df['email'].astype(str) + '_' + df['family_size'].astype(str) + '_' +\
        df['gender'].astype(str) + '_' + df['car'].astype(str) + '_' +\
        df['reality'].astype(str) + '_' + df['income_type'].astype(str) + '_' +\
        df['edu_type'].astype(str) + '_' + df['family_type'].astype(str) + '_' +\
        df['house_type'].astype(str) + '_' + df['occyp_type'].astype(str)

    cols = ['child_num', 'DAYS_BIRTH', 'DAYS_EMPLOYED',]
    tmp_x_train.drop(cols, axis=1, inplace=True)
    tmp_x_valid.drop(cols, axis=1, inplace=True)
    tmp_x_test.drop(cols, axis=1, inplace=True)

    cat_columns = [c for (c, t) in zip(tmp_x_train.dtypes.index, tmp_x_train.dtypes) if t == 'O'] 
    num_columns = [c for c in tmp_x_train.columns if c not in cat_columns]

    YJ_transform = PowerTransformer(method='yeo-johnson')
    tmp_x_train['income_total'] = YJ_transform.fit_transform(tmp_x_train['income_total'].values.reshape(-1, 1))
    tmp_x_valid['income_total'] = YJ_transform.transform(tmp_x_valid['income_total'].values.reshape(-1, 1))
    tmp_x_test['income_total'] = YJ_transform.transform(tmp_x_test['income_total'].values.reshape(-1, 1))


    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    num_columns.remove("income_total")

    scaler = select_scaler("standard")
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    ode = OrdinalEncoder(cat_columns)
    tmp_x_train[cat_columns] = ode.fit_transform(tmp_x_train[cat_columns], target)
    tmp_x_valid[cat_columns] = ode.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns]  = ode.transform(tmp_x_test[cat_columns])

    tmp_x_train['ID'] = tmp_x_train['ID'].astype('int64')
    tmp_x_valid['ID'] = tmp_x_valid['ID'].astype('int64')
    tmp_x_test['ID'] = tmp_x_test['ID'].astype('int64')

# ---------------------------    
    return tmp_x_train, tmp_x_valid, tmp_x_test, cat_columns, num_columns


In [152]:
train_path = os.path.join(BASE_DIR, 'data', 'MDC14', 'train.csv')
test_path  = os.path.join(BASE_DIR, 'data', 'MDC14', 'test.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

data.fillna("NaN", inplace = True)
test.fillna("NaN", inplace = True)

label = data['credit'] 

data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'], inplace=True) 

le = LabelEncoder()
label = le.fit_transform(label)

y_test_pred = np.zeros((test.shape[0], le.classes_.shape[0]))

In [153]:
X_train, X_valid, Y_train, Y_valid = train_test_split(data, label, test_size=0.2)

In [154]:
X_train.shape, X_valid.shape

((21165, 18), (5292, 18))

In [155]:
X_train, X_valid, test, cat_cols, num_cols = preprocess(X_train, X_valid, test, label)

C:\Users\HwaLang\miniconda3\envs\hltorch\lib\site-packages\sklearn\preprocessing\_data.py:3237: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


Select standard Scaler


In [156]:
data.shape, label.shape

((26457, 18), (26457,))

In [157]:
X_train[cat_cols]

,gender,car,reality,income_type,edu_type,family_type,house_type,occyp_type,ID
0,1,1,1,1,1,1,1,1,1
1,2,2,1,2,1,2,1,2,2
2,2,2,1,3,1,3,1,3,3
3,2,1,2,1,1,4,1,4,4
4,1,2,1,1,1,2,1,5,5
...,...,...,...,...,...,...,...,...,...
21160,2,1,1,3,1,1,1,2,5626
21161,2,1,2,1,1,1,1,1,4978
21162,2,2,2,4,1,1,1,6,4906
21163,2,1,2,2,1,1,1,7,24


In [158]:
for cat in cat_cols:
    print(np.unique(X_train[cat]), len(np.unique(X_train[cat])))

[1 2] 2
[1 2] 2
[1 2] 2
[1 2 3 4 5] 5
[1 2 3 4 5] 5
[1 2 3 4 5] 5
[1 2 3 4 5 6] 6
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] 19
[   1    2    3 ... 7982 7983 7984] 7984


In [132]:
for cat in cat_cols:
    print(np.unique(X_valid[cat]), len(np.unique(X_valid[cat])))

[1 2] 2
[1 2] 2
[1 2] 2
[1 2 3 4 5] 5
[1 2 3 4 5] 5
[1 2 3 4 5] 5
[1 2 3 4 5 6] 6
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] 19
[  -1    1    3 ... 8083 8086 8088] 3052


In [133]:
for cat in cat_cols:
    print(np.unique(test[cat]), len(np.unique(test[cat])))

[1 2] 2
[1 2] 2
[1 2] 2
[1 2 3 4 5] 5
[1 2 3 4 5] 5
[1 2 3 4 5] 5
[1 2 3 4 5 6] 6
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] 19
[  -1    2    3 ... 8086 8088 8089] 4423


In [12]:
cat_cols.remove("ID")

In [13]:
num_cols.append("ID")

In [14]:
cat_indics = [X_train.columns.get_loc(_) for _ in cat_cols]

In [15]:
cat_dims = [len(np.unique(X_train[_])) for _ in cat_cols]

In [205]:
model = TabNetClassifier(
    n_d=64, n_a=64, n_steps=5,
    gamma=1.5, n_independent=2, n_shared=2,
#     cat_idxs=cat_indics,
#     cat_dims=cat_dims,
#     cat_emb_dim=np.full(8, 3),
    lambda_sparse=1e-3, momentum=0.3, clip_value=2.,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-2),
    scheduler_params = {"gamma": 0.95,
                     "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15
)

Device used : cuda


In [206]:
cat_dims

[2, 2, 2, 5, 5, 5, 6, 19]

In [207]:
X_train.keys()

Index(['gender', 'car', 'reality', 'income_total', 'income_type', 'edu_type',
       'family_type', 'house_type', 'work_phone', 'phone', 'email',
       'occyp_type', 'family_size', 'begin_month', 'before_EMPLOYED',
       'income_total_befofeEMP_ratio', 'before_EMPLOYED_m',
       'before_EMPLOYED_w', 'Age', 'DAYS_BIRTH_m', 'DAYS_BIRTH_w', 'EMPLOYED',
       'DAYS_EMPLOYED_m', 'DAYS_EMPLOYED_w', 'ability', 'income_mean', 'ID'],
      dtype='object')

In [208]:
model.fit(X_train=X_train.values, y_train=y_train,
          eval_set=[(X_train.values, y_train), (X_valid.values, y_valid)],
          eval_name=['train', 'valid'],
          max_epochs=100, patience=10)

AssertionError: You need the same number of rows between X_train (21165) and y_train (16932)

In [24]:
with open(r"C:\Users\HwaLang\Desktop\python\T academy\Kaggle_camp\credit_card_dacon\NNI\SVM\model\SVM_model0.8828.pkl", "rb") as f:
    SVM = pickle.load(f)

In [28]:
with open(r"C:\Users\HwaLang\Desktop\python\T academy\Kaggle_camp\credit_card_dacon\NNI\catboost\model\cat_model0.6756.pkl", "rb") as f:
    cat_model = pickle.load(f)

In [29]:
with open(r"C:\Users\HwaLang\Desktop\python\T academy\Kaggle_camp\credit_card_dacon\NNI\RF\model\RF_model0.7149.pkl", "rb") as f:
    RF_model = pickle.load(f)

In [51]:
with open(r"C:\Users\HwaLang\Desktop\python\T academy\Kaggle_camp\credit_card_dacon\NNI\LGBM\LGBM_parmas_7107.json") as f:
    LGB_params = json.load(f)

In [62]:
with open(r"C:\Users\HwaLang\Desktop\python\T academy\Kaggle_camp\credit_card_dacon\NNI\XGB\1st_sol_params_0.696406.json") as f:
    XGB_params = json.load(f)

In [61]:
LGB_params

{'max_depth': 17,
 'subsample': 0.922673956606362,
 'colsample_bytree': 0.5207419913680738,
 'lr': 0.1036444950112919,
 'scaler': 'standard'}

In [63]:
XGB_params

{'max_depth': 13,
 'subsample': 0.7391142820439047,
 'colsample_bytree': 0.5021343602405989,
 'eta': 0.10767423370018543,
 'scaler': 'minmax'}

In [58]:
RF_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 32,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 29,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 3486,
 'n_jobs': 4,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [59]:
SVM.get_params()

{'C': 2.4301716177485737,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'sigmoid',
 'max_iter': -1,
 'probability': True,
 'random_state': None,
 'shrinking': True,
 'tol': 3.010968297611586e-05,
 'verbose': False}

In [60]:
cat_model.get_params()

{'learning_rate': 0.11053624353417138,
 'depth': 5,
 'border_count': 207,
 'bagging_temperature': 1,
 'min_data_in_leaf': 5}

In [166]:
train_path = os.path.join(BASE_DIR, 'data', 'MDC14', 'train.csv')
test_path  = os.path.join(BASE_DIR, 'data', 'MDC14', 'test.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

data.fillna("NaN", inplace = True)
test.fillna("NaN", inplace = True)

label = data['credit'] 

data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'], inplace=True) 

le = LabelEncoder()
label = le.fit_transform(label)

y_test_pred = np.zeros((test.shape[0], le.classes_.shape[0]))

X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.2)

print(X_train.shape, X_test.shape)

X_train, X_test, test, cat_cols, num_cols = preprocess(X_train, X_test, test, label)

(21165, 18) (5292, 18)


C:\Users\HwaLang\miniconda3\envs\hltorch\lib\site-packages\sklearn\preprocessing\_data.py:3237: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


Select standard Scaler


In [168]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier, Pool

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)


val_scores = list()
# 결과 값들을 stacking 해야하기 때문에, (모델 개수, 샘플의 수, 3) 라는 차원으로 구성됩니다.
oof_train = np.zeros((5, X_train.shape[0], 3))
oof_test  = np.zeros((5, X_valid.shape[0], 3))

# oof_pred = np.zeros((test.shape[0], 3))

for i, (trn_idx, val_idx) in enumerate(skf.split(X_train, Y_train)):
    x_train, y_train = X_train.iloc[trn_idx, :], pd.DataFrame(Y_train).iloc[trn_idx,]
    x_valid, y_valid = X_train.iloc[val_idx, :], pd.DataFrame(Y_train).iloc[val_idx,]
    
    # 전처리
    
    # 모델 정의
    models = [RandomForestClassifier(**RF_model.get_params()),
              XGBClassifier(**XGB_params),
              LGBMClassifier(**LGB_params),
              SVC(**SVM.get_params())]
            
    for j, model in enumerate(models):
        # 모델 학습
        model.fit(x_train, y_train)

        # j번째 칸에 맞는 결과 담기.
        oof_train[j, val_idx,] += model.predict_proba(x_valid)
        oof_test[j, :,]        += model.predict_proba(X_valid) / n_splits
    
    cat = CatBoostClassifier(**cat_model.get_params())
    
    train_data = Pool(data=x_train, label=y_train, cat_features=cat_cols)
    valid_data = Pool(data=x_valid, label=y_valid, cat_features=cat_cols)
    
    cat.fit(train_data, eval_set=valid_data, 
          use_best_model=True, 
          early_stopping_rounds=100, 
          verbose=100)
    
    oof_train[4, val_idx,] += cat.predict_proba(x_valid)
    oof_test[4, :,]        += cat.predict_proba(X_test) / n_splits
    
    print(f'{i} Fold, ...')

C:\Users\HwaLang\miniconda3\envs\hltorch\lib\site-packages\ipykernel_launcher.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\HwaLang\miniconda3\envs\hltorch\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\HwaLang\miniconda3\envs\hltorch\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[17:25:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "scaler" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:25:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\HwaLang\miniconda3\envs\hltorch\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[LightGBM] [Warning] Unknown parameter: lr
[LightGBM] [Warning] Unknown parameter: scaler


C:\Users\HwaLang\miniconda3\envs\hltorch\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

[17:34:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "scaler" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:34:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\HwaLang\miniconda3\envs\hltorch\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[LightGBM] [Warning] Unknown parameter: lr
[LightGBM] [Warning] Unknown parameter: scaler


C:\Users\HwaLang\miniconda3\envs\hltorch\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0:	learn: 1.0378137	test: 1.0360951	best: 1.0360951 (0)	total: 64.8ms	remaining: 1m 4s
100:	learn: 0.7291197	test: 0.6896822	best: 0.6896560 (96)	total: 3.23s	remaining: 28.7s
200:	learn: 0.7140427	test: 0.6889339	best: 0.6885320 (168)	total: 6.73s	remaining: 26.7s
300:	learn: 0.6996668	test: 0.6885173	best: 0.6884580 (286)	total: 10.4s	remaining: 24.1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6884579759
bestIteration = 286

Shrink model to first 287 iterations.
4 Fold, ...


In [179]:
oof_test.shape

(5, 5292, 3)

In [181]:
new_train.shape, Y_train.shape

((21165, 15), (21165,))

In [182]:
new_test.shape, Y_test.shape

((5292, 15), (5292,))

In [180]:
new_train = pd.DataFrame(np.concatenate(oof_train, axis=1))
new_test  = pd.DataFrame(np.concatenate(oof_test, axis=1))

In [188]:
from sklearn.preprocessing import StandardScaler

val_scores = list()
oof_pred  = np.zeros((test.shape[0], 3))

for i, (trn_idx, val_idx) in enumerate(skf.split(new_train, Y_train)):
    x_train, y_train = new_train.iloc[trn_idx, :], Y_train[trn_idx]
    x_valid, y_valid = new_train.iloc[val_idx, :], Y_train[val_idx]
    
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train     = scaler.transform(x_train)
    x_valid     = scaler.transform(x_valid)
    new_x_test  = scaler.transform(new_test)

    # 모델 정의
    model = XGBClassifier(random_state=42, n_jobs=(n_cpus-1))
    
    # 모델 학습
    model.fit(x_train, y_train)

    # 훈련, 검증 데이터 log_loss 확인
    trn_logloss = log_loss(y_train, model.predict_proba(x_train))
    val_logloss = log_loss(y_valid, model.predict_proba(x_valid))
    print('{} Fold, train logloss : {:.4f}4, validation logloss : {:.4f}'.format(i, trn_logloss, val_logloss))
    
    val_scores.append(val_logloss)
    
    # 반드시 log의 역함수인 exp를 취해주세요.
    oof_pred += model.predict_proba(new_x_test) / n_splits
    

# 교차 검증 log loss 평균 계산하기
print('Cross Validation Score : {:.5f}'.format(np.mean(val_scores)))

C:\Users\HwaLang\miniconda3\envs\hltorch\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:56:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0 Fold, train logloss : 0.38124, validation logloss : 0.7267


ValueError: operands could not be broadcast together with shapes (10000,3) (5292,3) (10000,3) 

In [ ]:
submit.iloc[:, 1:] = oof_pred
submit